In [2]:
# Add script to include project root in sys.path

import sys
from pathlib import Path

# Add project root (one level up from notebooks/)
project_root = Path(__file__).resolve().parent.parent if "__file__" in globals() else Path.cwd().parent
sys.path.append(str(project_root))

In [3]:
# Import necessary libraries

import os
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import torch
import torch.nn.functional as F
from torchvision import transforms, models
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

from src.gradcam import GradCAM


In [4]:
# Define constants and configurations

DATA_DIR = Path("static/output")   
MODEL_PATH = Path("saved_models/resnet50_best.pt")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])
